### Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación

### Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones 2021
Búsqueda y Recomendación para Textos Legales

Mentor: Jorge E. Pérez Villella

# Práctico Análisis y Visualización

Integrantes:

### Objetivos:

Normalizar el corpus generado en el práctico anterior, teniendo en cuenta los siguientes aspectos:

* tokenización, 
* pasar a minúsculas, 
* separar puntuación, 
* stemming y lematización, 
* eliminar stopwords (o no), 
* eliminar las palabras con frecuencia menor a n. 

Analizar las palabras más frecuentes de todo el corpus, por fuero y de 5 documentos. Compararlo con el resultado obtenido en el ejercicio anterior. Se observa algún cambio significativo?

Hacer una explicación con ejemplos tomando algunas palabras al azar entre lo que es stemming y lemmatizing para entender que nos da cada uno de estos procesos y cual es conveniente utilizar en cada caso.

Opcional:

* Investigar que es Segmentación y compararlo con Tokenización. Ejemplificar con un documento.
* Investigar NER (Named Entity Recognition - Reconocimiento de Entitades Nombradas). Buscar las Entidadas Nombradas mas frecuentes en todo el corpus y por fuero. 


Fecha de Entrega: 4 de julio de 2021

In [1]:
#!conda install -y -c conda-forge spacy=3.0.6

In [1]:
!python -m spacy validate

✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.0.6) =================
ℹ spaCy installation:
/Users/coviedo/opt/anaconda3/envs/diplodatos-ayvd/lib/python3.6/site-packages/spacy

NAME              SPACY            VERSION                            
es_core_news_md   >=3.0.0,<3.1.0   3.0.0   ✔
en_core_web_sm    >=3.0.0,<3.1.0   3.0.0   ✔



In [2]:
#!python -m spacy download es_core_news_md

In [52]:
import pandas as pd
import spacy
import datetime
from spacy.tokens.doc import Doc
import pickle
from collections import Counter

import lib.nlp_cba as nlp_cba

In [4]:
print(spacy.__version__)

3.0.6


In [85]:
#Nombre con el cual se graba el data frame corpus de todas las sentencias. Cada sentencia es una fila
corpus_file_name = "corpus.csv"

#Nombre con el cual se graba el data frame que tiene el corpus sumarizado por fuero. Todo el texto de las sentencias de un
# fuero esta en una sola fila
agregated_corups_df_file_name = "agregated_corpus.csv"


load_from_pickle = True

In [65]:
print (f"El data frame corpus_df se carga desde el archivo {corpus_file_name}")
corpus_df = pd.read_csv(corpus_file_name)

El data frame corpus_df se carga desde el archivo corpus.csv


In [66]:
print (f"El data frame agregated_corups_df se carga desde el archivo {agregated_corups_df_file_name}")
agregated_corups_df = pd.read_csv(agregated_corups_df_file_name)

El data frame agregated_corups_df se carga desde el archivo agregated_corpus.csv


In [67]:
corpus_df.head()

,text,classifier
0,datos de la causa sede ciudad de córdoba. de...,Documentos/MENORES
1,unívoco 18900 fecha 04/04/2016 materia niñe...,Documentos/MENORES
2,13/03/2013 juzgado de la niñez juventud y vio...,Documentos/MENORES
3,los autos caratulados a. a. - denuncia...,Documentos/MENORES
4,juzg. de niñez adolescencia y violencia famil...,Documentos/MENORES


In [68]:
agregated_corups_df.head()

,text,classifier
0,datos de la causa sede ciudad de córdoba. de...,Documentos/MENORES
1,sala penal - tribunal superior protocolo de s...,Documentos/PENAL
2,auto número sesenta y seis córdoba cinco de...,Documentos/FAMILIA
3,sala laboral - tribunal superior protocolo de...,Documentos/LABORAL


## Nomalización de Texto

In [69]:
#Cargamos el lenguaje español en spacy
spacy_nlp = spacy.load("es_core_news_md") 
spacy_nlp.max_length = 5000000

Doc.set_extension('text_id', default=False, force=True)

In [70]:
#Vemos las STOP_WORDS que viene definidas por defecto
#spacy.lang.es.stop_words.STOP_WORDS

In [71]:
# Stop_words que no estan contempladas en Spacy y consideramos necesario sacarlas

customs_stop_words = ['y' , 'e' , 'a']

for custom_stop_word in customs_stop_words:
   spacy_nlp.vocab[custom_stop_word].is_stop = True 

Clases para filtrar y transformar datos en Spacy

In [72]:
class SpacyTextNormalizer:
    def __init__(self):
        
        self.filters = []
        self.transformers = []
        self.documents = []
        
    def addFilter(self, filter):
        self.filters.append(filter)
        
    def addTransformer(self, transformer):
        self.transformers.append(transformer)
        
    
    def fit(self , spacy_tuples):
        self.documents = []
        for doc,context in spacy_tuples:
            doc._.text_id = context["text_id"]
            
            self.documents.append(doc)
        
        
    def normalize(self, filters = [], transformers = []):
        documents = []
        for doc in self.documents:
            words = []
            
            for word in doc:
                include = True

                for filter in filters:
                    include = filter.execute(word)
                    if not include: 
                        break
        
                if include:
                    
                    transformed_res = []
                    transformed_res.append(word)
                    
                    for transformer in transformers:
                        transformed_res.append(transformer.transform(word))
                        
                    words.append(transformed_res)       
        
            documents.append(( doc ,words))
        
        return documents

In [73]:
class MapMultipleCharsProcessor:

    def process(self, text , replace_chars):
        
        for ch in replace_chars:
            text = text.replace(ch[0],ch[1])

        
        return text


In [74]:
chars_replace = [('á' , 'a') , ('é' , 'e') , ('í', 'i') , ('ó' , 'o') , ('ú' , 'u')]
mapMultipleCharsProcessor = MapMultipleCharsProcessor()
mapMultipleCharsProcessor.process("Holá ú esté" , chars_replace)

'Hola u este'

In [75]:


class RemoveStopWordsAndPuntctuationFilter:
    
    def execute(self, word):
        return not word.is_stop and not word.is_punct


class RemoveSpaceFilter:
    def execute(self, word):
        return not word.is_space
    
    
class ToLowerCaseTransformer:
    def transform(self, word):
        return word.lower_
    
class ToLemaTransformer:
    def transform(self, word):
        return mapMultipleCharsProcessor.process(word.lemma_ , chars_replace)
    

Filtros y transformes que vamos a utilizar para normalizar los datos

In [76]:
#Filtros y transformers

removeSpaceFilter = RemoveSpaceFilter()
removeStopWordsAndPuntctuationFilter = RemoveStopWordsAndPuntctuationFilter ()


toLowerCaseTransformer = ToLowerCaseTransformer()
toLemaTransformer = ToLemaTransformer()



In [77]:
# Prueba de filtros y transformers
texto = [
    ("Pregunta?", {"text_id": "1"} ),
    ("Pregunta.", {"text_id": "1"} ),
    ("Estaba comiendo.", {"text_id": "1"} ),
    ("Muchos espacios en     blanco .", {"text_id": "1"} )
    ]

doc_tuples = spacy_nlp.pipe(texto , as_tuples=True , n_process=-1  )


spacyTextNormalizer = SpacyTextNormalizer()

spacyTextNormalizer.fit(doc_tuples)
result = spacyTextNormalizer.normalize(transformers=[toLemaTransformer])

In [78]:
#result[0][0]._.text_id
result

[(Pregunta?, [[Pregunta, 'Pregunta'], [?, '?']]),
 (Pregunta., [[Pregunta, 'Pregunta'], [., '.']]),
 (Estaba comiendo., [[Estaba, 'estar'], [comiendo, 'comer'], [., '.']]),
 (Muchos espacios en     blanco .,
  [[Muchos, 'mucho'],
   [espacios, 'espacio'],
   [en, 'en'],
   [    , '    '],
   [blanco, 'blanco'],
   [., '.']])]

In [79]:
# Extremadamente lento este enfoque para el caso de procesar el copus por fuero. El length del string del texto del corpus es 'grande' y 
# hace que sea lento el procesamiento
# Código deprecado

if False:
    texto = [(agregated_corups_df.iloc[2,].text , {"text_id": "1"})]

    doc_tuples = spacy_nlp.pipe(texto , as_tuples=True ,batch_size=50, n_process=4 , disable=["tok2vec", "tagger",  "attribute_ruler"] )

    result = spacyTextNormalizer.normalize(doc_tuples)

In [80]:
#Armamos el array de tuplas a partir del data frame corpus_df. Usamos este data frame y no el agregated_corups_df puesto
# que el array de texto por tupla es muy grande y Spacy requiere más memoria ademas de se notablemente lento

if not load_from_pickle:

    texto = corpus_df.apply( lambda x : (x['text'] , {"text_id": "1"}) , axis=1)
    start_time = datetime.datetime.now()
    doc_tuples = spacy_nlp.pipe(texto , as_tuples=True ,batch_size=50, n_process=4  )

    spacyTextNormalizer.fit(doc_tuples)
    stop_time = datetime.datetime.now()

    print (f"Tiempo de procesamiento: {stop_time - start_time}")

Tiempo de procesamiento: 0:02:02.884425


In [81]:
# Poniendo a True serealizamos la instancia spacyTextNormalizer. Esto no permite reconstruir el objeto por medio del
# archivo serealizado. Levantar el archivo y recrear el objeto es mucho más rápido de recostruir el objeto usando nlp.pipe y fit
if True:
    filehandler = open("normilizer.pkl", 'wb') 
    pickle.dump(spacyTextNormalizer, filehandler)

In [82]:
#Recreamos el objeto spacyTextNormalizer desde un archivo, ver la explicación del punto anterior
if load_from_pickle:
    
    start_time = datetime.datetime.now()
    filehandler = open("normilizer.pkl", 'rb') 
    spacyTextNormalizer = pickle.load(filehandler)
    stop_time = datetime.datetime.now()  
    
    print (f"Tiempo de procesamiento: {stop_time - start_time}")

In [83]:
start_time = datetime.datetime.now()

filters = [removeSpaceFilter,removeStopWordsAndPuntctuationFilter]

transformers = [toLemaTransformer]

result = spacyTextNormalizer.normalize(filters=filters, transformers=transformers)

stop_time = datetime.datetime.now()

print (f"Cantidad de documentos: {len(result)}")
print (f"Tiempo de procesamiento: {stop_time - start_time}")

Cantidad de documentos: 243
Tiempo de procesamiento: 0:00:04.007877


In [96]:
result[242][1]


def get_word_count(document ):
    words = [ word[1] for word in document]
    counter = Counter (words)
    return pd.DataFrame(data = counter.most_common()  , columns= [ "word" , "count"])
 



result1 = get_word_count(result[1][1])
result1['document_id'] = 1

#result1




In [102]:
value = 0

for document in result:
    res = get_word_count(document[1])
    res['id'] = 1
    print (res)
    value = value +1 
    if value == 2:
        break
        

              word  count  id
0               m.     29   1
1              ley     24   1
2        violencia     23   1
3           medida     22   1
4         familiar     20   1
..             ...    ...  ..
659      siguiente      1   1
660      confirmar      1   1
661  protocolicese      1   1
662        hagasar      1   1
663           dese      1   1

[664 rows x 3 columns]
              word  count  id
0             niño     46   1
1               fs     31   1
2              art     29   1
3           medida     28   1
4          derecho     27   1
..             ...    ...  ..
917  protocolicese      1   1
918        hagasar      1   1
919           dese      1   1
920            fdo      1   1
921        wallace      1   1

[922 rows x 3 columns]


## Prueba Varias

In [ ]:
#Guardamos información de contexto del documento
spacy.tokens.doc.Doc.set_extension('text_id' , default =False , force = True)
doc_tuples = spacy_nlp.pipe([("Hola esto es una prueba que te parece" , {"text_id" : "text_id1_1"})] , as_tuples=True , n_process=-1)
docs = []

for doc, context in doc_tuples:
    doc._.text_id = context["text_id"]
    docs.append(doc)

In [ ]:
for doc in docs:
    for token in doc:
        print(f"ddd.{token.text} {token.is_sent_start}  {token.lemma_} {token.norm_} {token.pos_}{doc._.text_id} ")